In [122]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.font_manager
import seaborn as sns
%matplotlib inline
from matplotlib.pyplot import figure
import plotly.express as px
from scipy import stats
import plotly.graph_objects as go

In [ ]:
from google.colab import drive
drive.mount ('/content/drive')

In [ ]:
%load_ext google.colab.data_table

In [ ]:
shark_attack = pd.read_csv("/content/drive/MyDrive/RawData/attacks.csv", encoding='latin-1')
shark_attack.loc[0:,'Date':'Species ']

In [ ]:
# columns of my dataset
shark_attack.columns

In [127]:
shark_attack.rename(columns={'Species ': 'Species'}, inplace=True)

In [ ]:
species = shark_attack ['Species'].unique()
for s in species:
    print(s)

In [129]:
# Create a List of known shark species
list_of_species = ["White shark", "Tiger shark", "Lemon shark", "Bull shark", "Grey reef shark", "Tawny nurse shark", "Wobbegong shark", "Blacktip shark", "Galapagos shark", "Oceanic whitetip shark", "Cookiecutter shark", "Spinner shark", "Angel shark", "Dogfish shark", "Bronze whaler shark", "Reef shark", "Hammerhead shark", "Thresher shark", "Spurdog", "Lesser spotted dogfish", "Longfin mako shark", "Shortfin mako shark", "Goblin shark", "Soupfin shark", "Leopard shark", "Porbeagle", "Seven-gill shark", "Raggedtooth shark", "Sandtiger shark", "Cow shark", "Whitetip reef shark", "Salmon shark"]

In [130]:
# Remove rows whose "Species" is not in the list of shark species
shark_attack_species = shark_attack[shark_attack["Species"].isin(list_of_species)]

In [ ]:
# Question 1: What are the most dangerous types of sharks to humans?
# Assuming the sharks with most attacks are the ones who are most dangerous to people.

# species that attack most
species_attack = shark_attack_species.groupby('Species')['Species'].count().sort_values(ascending=False)[1:10]

# Create a bar chart
data = go.Bar(x = species_attack.index,y=species_attack.values,text=species_attack.values, marker_color='pink')

layout = go.Layout(title = 'Shark attack by species',
                   xaxis=dict(title='Species'),
                   yaxis=dict(title='Number of attacks'),
                  )
fig = go.Figure(
    data=data,
    layout=layout
)
fig.show()

In [ ]:
# Question 2: Are children more likely to be attacked by sharks?

# Unique values Age column
shark_attack['Age'].unique()

In [133]:
# Create a function to extract numeric values from a string
def extract_numeric_age(age_str):
    try:
        # Try to extract a numeric value from the string
        age = float(age_str)
        return age
    except ValueError:
        # If it fails, return NaN
        return np.nan

In [134]:
# Apply the function to the "Age" column and create "NumericAge"
shark_attack['NumericAge'] = shark_attack['Age'].apply(extract_numeric_age)
# Filter the rows in which numerical ages are present
shark_attack = shark_attack.dropna(subset=['NumericAge'])

In [ ]:
# Unique values Age column
shark_attack['Age'].unique()

In [ ]:
# To answer the question, a column has been added to distinguish between shark attacks on children (in this case under 18 years old) and adults (18 years and older).
# Adding column Child

shark_attack['Child'] = shark_attack['Age'] < '18'

# For visualization purposes, adjust the position of the column 'Child'
shark_attack.insert(11, 'Child', shark_attack.pop('Child'))
shark_attack.loc[0:,'Date':'Child']

In [ ]:
# Ultimately, the graph shows that the number of attacks on children is much lower than the number of attacks on people aged 18 and over.

# Create a bar chart of the 'Child' column
attack_on_children = shark_attack['Child'].value_counts()
attack_on_children.plot(kind='bar')

# Labels graph
plt.xlabel('Attacks on Child')
plt.ylabel('Number')
plt.xticks([0, 1], ['False', 'True'])

plt.show()

In [ ]:
# Question 3: Are shark attacks where sharks were provoked more or less dangerous?
# I want to see a total number of attacks in my line graph and add the numbers of provoked attacks and unprovoked attacks to see a clear difference.
# In the graph you can see a big difference between unprovoced attacks and provoced attacks.
# Given the large difference between provoked and unprovoked attacks, I would assume that even if you don't provoke an attack, you still have a high chance of being attacked by a shark.
# Not to mention the fact that I wouldn't provoke a shark to attack anyway.

# To make the graph easier to read, only data from 1800 onwards is shown. Few attacks took place before this.
shark_attack = shark_attack[shark_attack['Year'] >= 1800]

# Calculate the number of attacks per year
year = shark_attack['Year'].value_counts().sort_index()

# Calculate the number of 'Provoked' attacks per year
provoked_attacks = shark_attack[shark_attack['Type'] == 'Provoked']['Year'].value_counts().sort_index()

# Calculate the number of 'Unprovoked' attacks per year
unprovoked_attacks = shark_attack[shark_attack['Type'] == 'Unprovoked']['Year'].value_counts().sort_index()

# Make a line graph
plt.figure(figsize=(20, 6))

plt.plot(year.index, year.values, 'r-', label='Total Attacks')
plt.plot(provoked_attacks.index, provoked_attacks.values, 'y-', label='Provoked attacks')
plt.plot(unprovoked_attacks.index, unprovoked_attacks.values, 'g-', label='Unprovoked attacks')

plt.xlabel('Year')
plt.ylabel('Total Attacks')
plt.title('Attacks since 1800')

plt.legend()

plt.show()

In [ ]:
# Unique values Activity column
shark_attack['Activity'].unique()

In [ ]:
# Question 4: Are certain activities more likely to result in a shark attack?

# To gain a clear insight, i've made a top 10 of all the provoced attacks with the associated activity
attack_activity = shark_attack[shark_attack.Type == 'Provoked'].groupby('Activity')['Activity'].count().sort_values(ascending=False)[:10]
attack_activity

In [ ]:
# From the top 10 the activities that are essentially the same can be grouped
shark_attack['Activity'] = shark_attack['Activity'].str.replace('.*Fishing.*', 'Fishing', case=False, regex=True)
shark_attack['Activity'] = shark_attack['Activity'].str.replace('.*Diving.*', 'Diving', case=False, regex=True)
shark_attack['Activity'].unique()

In [ ]:
# Again a top 10 of all the provoced attacks with the associated activity
attack_activity = shark_attack[shark_attack.Type == 'Provoked'].groupby('Activity')['Activity'].count().sort_values(ascending=False)[:10]
attack_activity

In [143]:
#  I want to refrase the output for the last 2 in my top 10
shark_attack['Activity'] = shark_attack['Activity'].replace({"Moving shark from tuna vessel when boat rolled, placing both man & shark in chest-deep water": "Moving a shark",
                                         "On expedition filming a feature movie & standing on tripod": "Filming a shark"})

In [ ]:
# Again a top 10 of all the provoced attacks with the associated activity
attack_activity = shark_attack[shark_attack.Type == 'Provoked'].groupby('Activity')['Activity'].count().sort_values(ascending=False)[:10]
attack_activity

In [ ]:
# Create a bar chart to show activities that may pose the greatest risk of being attacked by a shark
fig = px.bar(attack_activity, x=attack_activity.values, y=attack_activity.index, orientation='h', labels={'index':'','x':'Attack Count'},
            title = 'Which activity creates the greatest risk of shark attack')
fig.update_layout(height=500, width=1000)
fig.show()